In [1]:
import sys
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.optim import Adam
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
import os
import numpy as np
import pandas as pd
from PIL import Image
from scipy.misc import imread
from torch.utils.data import Dataset
from torchvision.transforms import Compose, ToTensor, Normalize
import matplotlib.pyplot as plt

In [2]:
class CustomDataset(Dataset):
    def __init__(self, datadir, labels):
        self.labels = labels
        self.x_img = []
        self.y_img = []
        files = sorted(os.listdir(datadir))
        for fx in files:
            if fx.split('_')[-1] == 'sat.jpg':
                fy = '%s_mask.png' % (fx.split('_')[0])
                if fy in files:
                    self.x_img.append(os.path.join(datadir,fx))
                    self.y_img.append(os.path.join(datadir,fy))
        self.len = len(self.x_img)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        transform = Compose([ToTensor(), Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
        x = transform(imread(self.x_img[idx]))
        y = self.__img2label(imread(self.y_img[idx]))
        return {'x':x, 'y':y}

    def __img2label(self, img):
        output = np.zeros((img.shape[0], img.shape[1]), dtype=np.int64)
        for i,l in enumerate(reversed(self.labels)):
            output += ((img[:,:,0]==l[0]) & (img[:,:,1]==l[1]) & (img[:,:,2]==l[2]))*i
        return output

In [3]:
# Testing "CustomDataset" class
labels = [[0,255,255],[255,255,0],[255,0,255],[0,255,0],[0,0,255],[255,255,255],[0,0,0]]
n_classes = len(labels)

train_dataset = CustomDataset('hw3-train-validation/train/', labels)
#print(train_dataset[10]['x'].shape,'\n')
#print(train_dataset[10]['y'].shape)
temp_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=6)
for i,data in enumerate(temp_dataloader):
    if i==1: break
    print(data['x'].shape)
    print(data['y'].shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be 

torch.Size([8, 3, 512, 512])
torch.Size([8, 512, 512])


In [4]:
n_epochs = 50
labels = [[0,255,255],[255,255,0],[255,0,255],[0,255,0],[0,0,255],[255,255,255],[0,0,0]]
n_classes = len(labels)

train_dataset = CustomDataset('hw3-train-validation/train/', labels)
val_dataset = CustomDataset('hw3-train-validation/validation/', labels)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=8)
val_dataloader = DataLoader(val_dataset,batch_size=1, shuffle=False, num_workers=8)
print("Train data: %d" % len(train_dataset))
print("Val data: %d" % len(val_dataset))

Train data: 2313
Val data: 257


In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import vgg16, vgg16_bn


In [6]:
class FCN32s(nn.Module):
    def __init__(self, n_class):
        super(FCN32s, self).__init__()
        self.pad = nn.ZeroPad2d(100)

        base_model = vgg16(pretrained=True)
        print(base_model)
        modules = list(base_model.children())[0]
        self.base = nn.Sequential(*modules)

        score = [
            nn.Conv2d(512, 4096, (7,7)),
            nn.ReLU(True),
            nn.Dropout2d(0.5, True),
            nn.Conv2d(4096, 4096, (1,1)),
            nn.ReLU(True),
            nn.Dropout2d(0.5, True),
            nn.Conv2d(4096, n_class, (1,1)),
            nn.ConvTranspose2d(n_class, n_class, (64,64), 32)]
        self.score = nn.Sequential(*score)

    def forward(self, input):
        x = self.pad(input)
        x = self.base(x)
        x = self.score(x)
        offset_x = (x.shape[2]-input.shape[2])//2
        offset_y = (x.shape[3]-input.shape[3])//2
        output = x[:,:,offset_x:offset_x+input.shape[2],offset_y:offset_y+input.shape[3]]
        return output


In [7]:
model = FCN32s(n_classes).cuda()
optimizer = Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

VGG(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_size=(3, 3), 

In [8]:
print(model)

FCN32s(
  (pad): ZeroPad2d((100, 100, 100, 100))
  (base): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17)

In [9]:
def train(data_loader, model, optimizer, criterion):
    train_acc = []
    train_loss = []
    pbar = tqdm(total=len(train_dataloader))
    for data in data_loader:
        x = data['x'].cuda()
        y = data['y'].cuda()

        optimizer.zero_grad()
        pred_y = model(x)
        loss = criterion(pred_y, y)
        loss.backward()
        optimizer.step()
        
        acc = cal_acc(pred_y.data, y.data)
        pbar.set_description('loss: %.3f acc: %.3f' % (loss.item(),acc))
        pbar.update(1)
        train_acc.append(acc)
        train_loss.append(loss.item())
    pbar.close()
    return np.mean(train_loss), np.mean(train_acc)

In [10]:
def val(data_loader, model, criterion):
    model.train(False)
    val_acc = []
    val_loss = []
    with torch.no_grad():
        for data in data_loader:
            x = data['x'].cuda()
            y = data['y'].cuda()

            pred_y = model(x)
            loss = criterion(pred_y, y)
            val_acc.append(cal_acc(pred_y.data, y.data))
            val_loss.append(loss)

    model.train(True)
    return np.mean(val_loss), np.mean(val_acc)

In [11]:
def cal_acc(pred_y, true_y):
    _, argmax = torch.max(pred_y, 1)
    return argmax.eq(true_y).float().mean()

In [12]:
for epoch in range(1, n_epochs+1):
    print('epoch %d/%d' % (epoch, n_epochs))
    # Train
    train_loss, train_acc = train(train_dataloader, model, optimizer, criterion)

    # Val
    val_loss, val_acc = val(val_dataloader, model, criterion)
    print('val loss: %.3f val acc: %.3f' % (val_loss, val_acc))

    '''
    record = {'train_acc': train_acc, 'train_loss': train_loss,
              'val_acc': val_acc, 'val_loss': val_loss}
    logger.log(record, epoch)
    if val_acc > best_acc:
        torch.save(model, 'best_model_ep%d_%.3f.pth' % (epoch, val_acc))
        best_acc = val_acc
    if epoch % 5 == 0:
        torch.save(model, 'model_ep%d_%.3f.pth' % (epoch, val_acc))
    '''

  0%|          | 0/290 [00:00<?, ?it/s]

epoch 1/50


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be 

  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt


KeyboardInterrupt: 

0.3.0.post4
